In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sklearn.decomposition import PCA
import numpy as np
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from scipy.special import expit, logit
import pandas as pd

def One_Hot_Encoding(y, n_classes):
    arr = np.zeros((y.shape[0],n_classes))
    for i in range(y.shape[0]):
        e = [0.0]*n_classes
        e[y[i]] = 1.0
        arr[i] = e
    return arr


def Get_Data(file, data, normalise = True, mean=0, std=1):
    print('Loading Data...')
    n_classes = 10
    
    df = pd.read_csv(file)
    
    df = df.iloc[np.random.permutation(len(df))]
    
    ##img_id = df['id']
    
    X = df[df.columns[0:-1]]
  
    
    if data == 'train':
        mean = X.mean(axis=0)
        std  = X.std(axis = 0)
    
    if normalise == True:
      X = normaliseData(X, mean, std)
    
    X = X.values
  
    Y = df['label'].values
    
    
    print('Data Loaded Successfully...')
    
    return (X, One_Hot_Encoding(Y, n_classes), mean, std)
def normaliseData(X, mean, std):
    X = X - mean
    X = X/std
    return X




(X_train, Y_train, train_mean, train_std)=Get_Data('fashion-mnist_train.csv', data = 'train', normalise = True )

X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)
(X_Test, Y_Test, train_mean, train_std)=Get_Data('fashion-mnist_test.csv', data = 'train', normalise = True )

print(X_Test.shape,Y_Test.shape)


Loading Data...
Data Loaded Successfully...
Loading Data...
Data Loaded Successfully...
(10000, 784) (10000, 10)


In [2]:
import wandb
from re import X
import numpy as np
import warnings
from sklearn.metrics import log_loss
from scipy.special import expit, logit
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
class Feed_Forward_NeuralNetwork(object):
  number_of_neurons=[]
  number_of_layers=0
  Biases=[]
  Weights=[]
  Activation_gx=[]
  Layer_Output=[]
  Layer_Input=[]
  Loss_Function=""

  def __init__(self,Layers,Activations,Loss_Funciton="CE",seed=8790,Weight_init='Random'):
    self.number_of_layers=len(Layers)
    self.number_of_neurons=Layers
    self.Activation_gx=Activations
    self.Loss_Function=Loss_Funciton
    if Weight_init=='Xavier':
      self.Weights=[np.random.randn(sizej,sizei)*np.sqrt(1/(sizei)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]  
    else:
      self.Weights=[np.random.randn(sizej,sizei)*np.sqrt(2/(sizei+sizej)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]
    self.Biases=[np.zeros((x,1)) for x in self.number_of_neurons[1:]]
    


  def Sigmoid(self,x):
    return  expit(x)
  def stable_softmax(self,x):
    out = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
      exps = np.exp(x[:, i] - np.max(x[:, i]))
      out[:, i] = exps / np.sum( exps)
    return out
  def Sigmoid_Drv(self, x):
    sigma_x = self.Sigmoid(x)
    return (sigma_x * (1 - sigma_x))
  def Relu(self,x):
    return np.maximum(0,x,x)
  def Tanh(self,x):
    return np.tanh(x)
  def Tanh_Drv(self, x):
    tanh_x = np.tanh(x)
    return (1 - np.square(tanh_x))
  def Relu_Drv(self, x):
    return np.greater(x,0).astype(int)


  def Feed_Network(self,X_train):
    self.Layer_Input=[]
    self.Layer_Output=[]
    Weights=self.Weights
    biases=self.Biases
    no_of_datapoints=X_train.shape[0]
    no_of_features=X_train.shape[1]
    Temp=[]
    Activations=[]

    for i in range(0,self.number_of_layers-1):

      if i==0:
        Temp.append(np.matmul(Weights[i],X_train.T)+biases[i])
      else:
        Temp.append(np.matmul(Weights[i],Activations[i-1])+biases[i])
      
      self.Layer_Input.append(Temp[i])


      if (self.Activation_gx[i]=='SoftMax') and (i == self.number_of_layers-2):
        Activations.append(self.stable_softmax(Temp[i]))
      elif self.Activation_gx[i]=='Tanh':
        Activations.append(self.Tanh(Temp[i]))
      elif self.Activation_gx[i]=='Sigmoid':
        Activations.append(self.Sigmoid(Temp[i]))
      elif self.Activation_gx[i]=='Relu':
        Activations.append(self.Relu(Temp[i]))
      



      self.Layer_Output.append(Activations[i])
    return self.Layer_Output[-1]

  def Backward_Pass(self,X_train,Y_train):
    Gredient_wrt_weight=[0]*(len(self.Weights))
    Gradient_wrt_biases=[0]*(len(self.Biases))
    Gradient_wrt_layers=[0]*(self.number_of_layers-1)
    n=X_train.shape[0]


    for i in reversed(range(self.number_of_layers-1)):
      if i == self.number_of_layers-2:
        if self.Loss_Function=='CE':
          Gradient_wrt_layers[i]=self.Layer_Output[i]-Y_train.T
        elif self.Loss_Function=='MSE':
          Gradient_wrt_layers[i]=(self.Layer_Output[i]-Y_train.T)*self.Layer_Output[i]*(1-self.Layer_Output[i])

        Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],self.Layer_Output[i-1].T)
        Gradient_wrt_biases[i]=(1/n)*np.sum(Gradient_wrt_layers[i],axis=1,keepdims=True)
      else:
        activation_derivative=[]
        if self.Activation_gx[i]=='Tanh':
          activation_derivative=self.Tanh_Drv(Gradient_wrt_layers[i])
        elif self.Activation_gx[i]=='Sigmoid':
          activation_derivative=self.Sigmoid_Drv(Gradient_wrt_layers[i])
        elif self.Activation_gx[i]=='Relu':
          activation_derivative=self.Relu_Drv(Gradient_wrt_layers[i])
        
        Gradient_wrt_layers[i]=np.matmul(self.Weights[i+1].T,Gradient_wrt_layers[i+1])*activation_derivative#self.Tanh_Drv(Gradient_wrt_layers[i])
        
        if i==0:
          Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],X_train)
        else:
          Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],self.Layer_Output[i-1].T)
        
        Gradient_wrt_biases[i]=(1/n)*np.sum(Gradient_wrt_layers[i],axis=1,keepdims=True)
    return (Gradient_wrt_biases,Gredient_wrt_weight,Gradient_wrt_layers)

  def Get_Accuracy(self,y,y_pred):
    P=np.argmax(y,axis=0)
    Q=np.argmax(y_pred,axis=0)
    acc=np.sum(P == Q)/len(P)
    return 100*acc
  def GD(self,Grd_w,Grd_b,Eta):
    return (np.multiply(Eta,Grd_w),np.multiply(Eta,Grd_b))
  def Momentum(self,Momentum,Grd_w,Grd_b,Eta,prev_w,prev_b):
    upd_weight=np.multiply(Momentum,prev_w)+np.multiply(Eta,Grd_w)
    upd_biases=np.multiply(Momentum,prev_b)+np.multiply(Eta,Grd_b)
    return (upd_weight,upd_biases)
  def RMSprop(self,Grd_w,Grd_b,Eta):
    w_v,b_v=0,0
    eps=1e-3
    beta=0.9
    w_v=np.multiply(beta,w_v)
    b_v=np.multiply(beta,b_v)
    w_v=w_v+np.multiply(1-beta,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta,np.power(Grd_b,2))
    w_v_corrected=1/(np.power(w_v+eps,0.5))
    b_v_corrected=1/(np.power(b_v+eps,0.5))
    upd_w=np.multiply(w_v_corrected,Grd_w)*Eta
    upd_b=np.multiply(b_v_corrected,Grd_b)*Eta
    return (upd_w,upd_b)
  def Adam(self,Grd_w,Grd_b,Eta,t):
    w_m,b_m=0,0
    w_v,b_v=0,0
    beta1,beta2=0.9,0.99
    eps=1e-8
    w_v=np.multiply(beta2,w_v)
    b_v=np.multiply(beta2,b_v)
    w_v=w_v+np.multiply(1-beta2,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta2,np.power(Grd_b,2))
    w_v=w_v/(1-np.power(beta2,t+1))
    b_v=b_v/(1-np.power(beta2,t+1))
    w_m=np.multiply(beta1,w_m)
    b_m=np.multiply(beta1,b_m)
    w_m=w_m+np.multiply(1-beta1,Grd_w)
    b_m=b_m+np.multiply(1-beta1,Grd_b)
    w_m=w_m/(1-np.power(beta1,t+1))
    b_m=b_m/(1-np.power(beta1,t+1))
    upd_w=(Eta/np.power(w_v+eps,0.5))*w_m
    upd_b=(Eta/np.power(b_v+eps,0.5))*b_m

    return (upd_w,upd_b)
  def Nadam(self,Grd_w,Grd_b,Eta,t):
    w_m,b_m=0,0
    w_v,b_v=0,0
    beta1,beta2=0.9,0.99
    eps=1e-8
    w_v=np.multiply(beta2,w_v)
    b_v=np.multiply(beta2,b_v)
    w_v=w_v+np.multiply(1-beta2,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta2,np.power(Grd_b,2))
    w_m=np.multiply(beta1,w_m)
    b_m=np.multiply(beta1,b_m)
    w_m=w_m+np.multiply(1-beta1,Grd_w)
    b_m=b_m+np.multiply(1-beta1,Grd_b)
    w_m=np.multiply(beta1,w_m)/(1-beta1)+np.multiply(beta1,Grd_w)/(1-beta1)
    b_m=np.multiply(beta1,b_m)/(1-beta1)+np.multiply(beta1,Grd_b)/(1-beta1)
    w_v=np.multiply(np.multiply(beta2,w_v),1/(1-beta2))
    b_v=np.multiply(np.multiply(beta2,b_v),1/(1-beta2))
    upd_w=(Eta/np.power(w_v+eps,0.5))*w_m
    upd_b=(Eta/np.power(b_v+eps,0.5))*b_m 
                                
    return (upd_w,upd_b)
  def Nag(self,gamma,Grd_w,Grd_b,Eta,prev_w,prev_b):
    w_v,b_v=0,0
    w_v=np.multiply(gamma,prev_w)+np.multiply(Eta,Grd_w)
    b_v=np.multiply(gamma,prev_b)+np.multiply(Eta,Grd_b)
    prev_w=w_v 
    prev_b=b_v
    w_v=np.multiply(gamma,prev_w)
    b_v=np.multiply(gamma,prev_b)
    return w_v,b_v






  def Main_Training_Algo(self,X_Valid,Y_Valid,X_Train,Y_Train,optimizer='Momentum',NEpochs=5,Batch_Size=16,Momentum=0.9,Eta=0.01,anneal=False):
    etap=Eta
    Loss_in_Validation=[]
    Loss_in_Training=[]
    update_weight=[]
    update_biases=[]
    prev_w=[np.zeros((sizej,sizei)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]
    prev_b=[np.zeros((x,1)) for x in self.number_of_neurons[1:]]
    t=0
    for Ep in range(NEpochs):
      Loss_in_batch=0

      for start in range(0,X_Train.shape[0],Batch_Size):

        X_Train_curr_batch=X_Train[start:start+Batch_Size]
        Y_Train_curr_batch=Y_Train[start:start+Batch_Size]
        

        Curr_Batch_Prediction=self.Feed_Network(X_Train_curr_batch)

        (Gradient_wrt_biases,Gradient_wrt_weight,Gradient_wrt_layers)=self.Backward_Pass(X_Train_curr_batch,Y_Train_curr_batch)


        if optimizer=='GD':
          update_weight,update_biases=self.GD(Gradient_wrt_weight,Gradient_wrt_biases,Eta)
        elif optimizer=='Momentum':
          update_weight,update_biases=self.Momentum(Momentum,Gradient_wrt_weight,Gradient_wrt_biases,Eta,prev_w,prev_b)
          prev_w=update_weight
          prev_b=update_biases
        elif optimizer=='RMSprop':
          update_weight,update_biases=self.RMSprop(Gradient_wrt_weight,Gradient_wrt_biases,Eta)
        elif optimizer=='Adam':
          update_weight,update_biases=self.Adam(Gradient_wrt_weight,Gradient_wrt_biases,Eta,t)
        elif optimizer=='Nadam':
          update_weight,update_biases=self.Nadam(Gradient_wrt_weight,Gradient_wrt_biases,Eta,t)
        elif optimizer=='Nag':
          update_weight,update_biases=self.Nag(Momentum,Gradient_wrt_weight,Gradient_wrt_biases,Eta,prev_w,prev_b)
          prev_w=update_weight
          prev_b=update_biases
          
        t=t+1

        
        self.Weights=self.Weights-update_weight
        self.Biases=self.Biases-update_biases
        if anneal == True:
          Eta = ((0.99)**(start+3.5))*etap

        if self.Loss_Function=='CE':
          Loss_in_batch=Loss_in_batch+log_loss(Y_Train_curr_batch.T,Curr_Batch_Prediction)
        elif self.Loss_Function=='MSE':
          Loss_in_batch=Loss_in_batch+(mean_squared_error(Y_Train_curr_batch.T,Curr_Batch_Prediction))          
      

      Loss_in_Training.append(Loss_in_batch)
      Train_Prediction=self.Feed_Network(X_Train)
      print("Epoch:",Ep)
      print("Train Accuracy",self.Get_Accuracy(Y_Train.T,Train_Prediction))
      print("Loss in Training",Loss_in_Training[Ep])
      
      Validation_Predcition=self.Feed_Network(X_Valid)
      if self.Loss_Function=='CE':
        Loss_in_Validation.append(log_loss(Y_Valid.T,Validation_Predcition))
      elif self.Loss_Function=='MSE':
        Loss_in_Validation.append(mean_squared_error(Y_Valid.T,Validation_Predcition))
      print("Validation Accuracy",self.Get_Accuracy(Y_Valid.T,Validation_Predcition))
      print("Loss in Validation",Loss_in_Validation[Ep])
      wandb.log({"val_loss_end": Loss_in_Training[Ep]/10000, \
                       "val_acc_end": self.Get_Accuracy(Y_Train.T,Train_Prediction), \
                       "test_loss_end": Loss_in_Validation[Ep]/10000, \
                       "test_acc_end": self.Get_Accuracy(Y_Valid.T,Validation_Predcition), \
                       "epoch":Ep})



# n_classes = 10
# wandb.init()
# loss_function = 'CE'
# seed =8790
# Activation_gx = ['Tanh', 'Tanh','Tanh','Tanh', 'SoftMax']     #Activation Fucntion used for hidden layer are {Tanh,Relu,Sigmoid}
# neurons = 128
# Layers = [784, neurons,neurons ,neurons,neurons, n_classes]
# #Weight_init="Xavier"
# Weight_init="Random"
# Network=Feed_Forward_NeuralNetwork(Layers,Activation_gx,loss_function,seed,Weight_init)
# #optimizer used are {Momentum,Nag,Adam,Nadam,GD,RMSprop}
# print(Network.Main_Training_Algo(X_Valid,Y_Valid,X_Train,Y_Train,optimizer='Momentum',NEpochs=10,Batch_Size=16,Momentum=0.9,Eta=0.001,anneal=True))
# Y_test_pred=Network.Feed_Network(X_Test)
# print("Test Acc:", Network.Get_Accuracy(Y_Test.T,Y_test_pred))
# print("Loss In Test",log_loss(Y_Test.T,Y_test_pred))

    



In [3]:
def only():
    sweep_config = {"name": "complete-sweep", "method": "grid"}
    sweep_config["metric"] = {"name": "accuracy", "goal": "minimize"}
    parameters_dict = {
                    "num_epochs": {"values": [10]}, \
                     "num_hidden_layers": {"values": [4]}, \
                    "size_hidden_layer": {"values": [32, 64, 128]}, \
                    "learning_rate": {"values": [1e-3, 1e-4]}, \
                    "optimizer": {"values": ["GD","Momentum","Nag","RMSProp","Adam","Nadam"]}, \
                    "batch_size": {"values": [160, 320, 640]}, \
                    "weight_init": {"values": ["Random", "Xavier"]} , \
                    "activation": {"values": ["Tanh", "Relu","Sigmoid"]}, \
                    "loss": {"values": ["CE", "MSE"]}, \
                      }
    sweep_config["parameters"] = parameters_dict
    global accuracy
    accuracy=0
    def train_nn(config = sweep_config):
        global accuracy
        with wandb.init(config = config):
            config = wandb.init().config
            wandb.run.name = "e_{}_hl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.num_epochs,\
                                                                          config.size_hidden_layer,\
                                                                          config.optimizer,\
                                                                          config.batch_size,\
                                                                          config.weight_init,\
                                                                          config.activation,\
                                                                          accuracy)






            n_classes = 10
            seed =8790
            loss_function = config.loss
            Activation_gx = [config.activation, config.activation,config.activation,config.activation, 'SoftMax']
            neurons =config.size_hidden_layer
            Layers = [784, neurons,neurons ,neurons,neurons, n_classes]
            Network=Feed_Forward_NeuralNetwork(Layers,Activation_gx,loss_function,seed,config.weight_init) 
            Network.Main_Training_Algo(X_Valid,Y_Valid,X_Train,Y_Train,config.optimizer,config.num_epochs,config.batch_size,config.learning_rate,0.001,anneal=True)
            Validation_Predcition=Network.Feed_Network(X_Valid)
            accuracy=Network.Get_Accuracy(Y_Valid.T,Validation_Predcition)

            wandb.log({"accuracy": accuracy})
            #wandb.log({"accuracy": accuracy, "validation_accuracy": val_acc, "training_loss": cost, "validation cost": val_cost, 'epoch': count})

    sweep_id = wandb.sweep(sweep_config, project = "Detail-1")
    wandb.agent(sweep_id, function = train_nn)





In [ ]:
only()

Create sweep with ID: e8k0srji
Sweep URL: https://wandb.ai/vishal91099/Detail-1/sweeps/e8k0srji


wandb: Agent Starting Run: o87zfz7z with config:
wandb: 	activation: Tanh
wandb: 	batch_size: 160
wandb: 	learning_rate: 0.001
wandb: 	loss: CE
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: GD
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: RandomNormal
wandb: Currently logged in as: vishal91099 (use `wandb login --relogin` to force relogin)


Epoch: 0
Train Accuracy 7.275925925925926
Loss in Training 27930.001621683732
Validation Accuracy 6.9
Loss in Validation 5283.677280955681
Epoch: 1
Train Accuracy 7.546296296296297
Loss in Training 27875.03117257285
Validation Accuracy 7.35
Loss in Validation 5277.478024584563
Epoch: 2
Train Accuracy 7.842592592592593
Loss in Training 27820.788206385383
Validation Accuracy 7.6499999999999995
Loss in Validation 5271.3428833969365
Epoch: 3
Train Accuracy 8.15
Loss in Training 27767.133345128936
Validation Accuracy 7.983333333333334
Loss in Validation 5265.273677949695
Epoch: 4
Train Accuracy 8.437037037037037
Loss in Training 27714.08025243628
Validation Accuracy 8.316666666666666
Loss in Validation 5259.272081068377
Epoch: 5
Train Accuracy 8.698148148148148
Loss in Training 27661.64129837145
Validation Accuracy 8.566666666666666
Loss in Validation 5253.339605805866
Epoch: 6
Train Accuracy 9.007407407407406
Loss in Training 27609.82746680383
Validation Accuracy 8.866666666666667
Loss in 

wandb: Agent Starting Run: kdaz9nxu with config:
wandb: 	activation: Tanh
wandb: 	batch_size: 160
wandb: 	learning_rate: 0.001
wandb: 	loss: CE
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: GD
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: Xavier


Epoch: 0
Train Accuracy 15.838888888888889
Loss in Training 27188.817701257125
Validation Accuracy 15.916666666666668
Loss in Validation 5194.227643014232
Epoch: 1
Train Accuracy 16.596296296296295
Loss in Training 27135.537646756784
Validation Accuracy 16.633333333333333
Loss in Validation 5188.290620972315
Epoch: 2
Train Accuracy 17.353703703703705
Loss in Training 27083.97099650592
Validation Accuracy 17.349999999999998
Loss in Validation 5182.532938102949
Epoch: 3
Train Accuracy 18.022222222222222
Loss in Training 27033.92868764637
Validation Accuracy 18.05
Loss in Validation 5176.950300111303
Epoch: 4
Train Accuracy 18.75925925925926
Loss in Training 26985.369045363284
Validation Accuracy 18.71666666666667
Loss in Validation 5171.537624740253
Epoch: 5
Train Accuracy 19.461111111111112
Loss in Training 26938.245226685543
Validation Accuracy 19.28333333333333
Loss in Validation 5166.289271230415
Epoch: 6
Train Accuracy 20.053703703703704
Loss in Training 26892.506962567008
Validatio

...

Epoch: 0
Train Accuracy 10.583333333333334
Loss in Training 27322.36698942057
Validation Accuracy 11.1
Loss in Validation 5209.184070246649
Epoch: 1
Train Accuracy 10.585185185185185
Loss in Training 27322.287566107567
Validation Accuracy 11.1
Loss in Validation 5209.175284745655
Epoch: 2
Train Accuracy 10.587037037037037
Loss in Training 27322.208347200267
Validation Accuracy 11.1
Loss in Validation 5209.166499383619
Epoch: 3
Train Accuracy 10.588888888888889
Loss in Training 27322.129129553618
Validation Accuracy 11.1
Loss in Validation 5209.157714160545
Epoch: 4
Train Accuracy 10.59074074074074
Loss in Training 27322.04991316767
Validation Accuracy 11.1
Loss in Validation 5209.148929076425
Epoch: 5
Train Accuracy 10.59074074074074
Loss in Training 27321.97069804234
Validation Accuracy 11.1
Loss in Validation 5209.14014413126
Epoch: 6
Train Accuracy 10.592592592592592
Loss in Training 27321.89148417768
Validation Accuracy 11.1
Loss in Validation 5209.131359325049
Epoch: 7
Train Accur

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: pirv8rk8 with config:
wandb: 	activation: Tanh
wandb: 	batch_size: 160
wandb: 	learning_rate: 0.001
wandb: 	loss: CE
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: RMSProp
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: RandomNormal


wandb: ERROR Run pirv8rk8 errored: TypeError("unsupported operand type(s) for -: 'list' and 'list'",)
wandb: Agent Starting Run: uje26de0 with config:
wandb: 	activation: Tanh
wandb: 	batch_size: 160
wandb: 	learning_rate: 0.001
wandb: 	loss: CE
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: RMSProp
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: Xavier


Problem at: <ipython-input-3-bd06879fe522> 20 train_nn


Traceback (most recent call last):
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\wandb_init.py", line 951, in init
    run = wi.init()
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\wandb_init.py", line 597, in init
    ret = backend.interface.communicate_run(run, timeout=30)
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\interface\interface.py", line 210, in communicate_run
    return self._communicate_run(run, timeout=timeout)
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 311, in _communicate_run
    resp = self._communicate(req, timeout=timeout)
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 213, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "C:\Users\Manish\Anaconda3\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 218, in _communicate_async
    raise Exception("The w